# CAP 6776 HW3 - BM25, Evaluation, PR Curve, Error Analysis

This notebook follows the Homework 3 required outputs and file formats (see the assignment PDF).  
Important: you must replace the sample `data/corpus.csv` and `data/queries.csv` with your own corpus (min 400 docs) and your own queries.

About qrels:
- If `data/qrels.csv` is missing, the notebook will generate a starter pool from the default run (top-20 per query).
- Then you edit `data/qrels.csv` and re-run Part B, C, D.

What this notebook generates:
- BM25 default run and tuned run
- P@10, MAP, nDCG@10
- Precision-Recall curve for one query
- Short error analysis text file



## Part 0 - Setup and Load Data

Required structure inside your ZIP:

- data/corpus.csv
- data/queries.csv
- data/qrels.csv
- outputs/run_bm25_default.csv
- outputs/run_bm25_tuned.csv
- outputs/metrics_summary.csv
- outputs/pr_curve_query_<query_id>.csv
- outputs/pr_curve_query_<query_id>.png
- outputs/error_analysis.txt
- hw3.ipynb


In [ ]:
import os
from pathlib import Path
import re, math
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
OUT_DIR = BASE_DIR / 'outputs'
OUT_DIR.mkdir(parents=True, exist_ok=True)

CORPUS_PATH = DATA_DIR / 'corpus.csv'
QUERIES_PATH = DATA_DIR / 'queries.csv'
QRELS_PATH = DATA_DIR / 'qrels.csv'

corpus_df = pd.read_csv(CORPUS_PATH)
queries_df = pd.read_csv(QUERIES_PATH)

print('Corpus rows:', len(corpus_df))
print('Queries:', len(queries_df))

if len(corpus_df) < 400:
    print('Warning: corpus has less than 400 docs. Replace the sample corpus with your real corpus.')

if QRELS_PATH.exists():
    qrels_df = pd.read_csv(QRELS_PATH)
    print('Qrels rows:', len(qrels_df))
else:
    qrels_df = pd.DataFrame(columns=['query_id','doc_id','relevance'])
    print('qrels.csv not found yet. I will generate a starter pool after Part A.')


## BM25 Implementation

Tokenizer: simple lowercase + alphanumeric terms.
If your corpus needs something else (like keeping hyphens, or using stemming), update `tokenize()`.


In [ ]:
def tokenize(text: str):
    return re.findall(r"[a-z0-9]+", str(text).lower())

class BM25:
    def __init__(self, corpus_texts, k1=1.2, b=0.75, epsilon=0.25):
        self.k1 = float(k1)
        self.b = float(b)
        self.epsilon = float(epsilon)

        self.docs = [tokenize(t) for t in corpus_texts]
        self.doc_lens = np.array([len(d) for d in self.docs], dtype=float)
        self.avgdl = float(np.mean(self.doc_lens)) if len(self.doc_lens) else 0.0
        self.N = len(self.docs)

        self.tfs = [Counter(d) for d in self.docs]
        df = Counter()
        for tf in self.tfs:
            for term in tf.keys():
                df[term] += 1
        self.df = df

        self.idf = {}
        idf_vals = []
        for term, n_q in self.df.items():
            idf = math.log(1 + (self.N - n_q + 0.5) / (n_q + 0.5))
            self.idf[term] = idf
            idf_vals.append(idf)

        self.idf_floor = self.epsilon * (float(np.mean(idf_vals)) if idf_vals else 0.0)

    def score(self, query_tokens):
        scores = np.zeros(self.N, dtype=float)
        qtf = Counter(query_tokens)

        for term in qtf.keys():
            idf = self.idf.get(term, self.idf_floor)
            for i, tf in enumerate(self.tfs):
                f = tf.get(term, 0)
                if f == 0:
                    continue
                denom = f + self.k1 * (1 - self.b + self.b * (self.doc_lens[i] / self.avgdl))
                scores[i] += idf * (f * (self.k1 + 1)) / denom

        return scores

def run_bm25(bm25, corpus_doc_ids, queries_df, topk=100):
    rows = []
    for _, q in queries_df.iterrows():
        qid = q['query_id']
        qtext = q['query_text']
        qtokens = tokenize(qtext)

        scores = bm25.score(qtokens)
        idx = np.argsort(-scores)

        rank = 0
        for j in idx[:topk]:
            if scores[j] <= 0:
                continue
            rank += 1
            rows.append((qid, corpus_doc_ids[j], rank, float(scores[j])))

    return pd.DataFrame(rows, columns=['query_id', 'doc_id', 'rank', 'score'])

def build_qrels_pool(run_df, topn=20):
    pool = []
    for qid, grp in run_df.groupby('query_id'):
        for _, r in grp.sort_values('rank').head(topn).iterrows():
            pool.append((qid, r['doc_id'], 0))
    return pd.DataFrame(pool, columns=['query_id','doc_id','relevance'])


## Part A - BM25 Ranking + Parameter Study

Default params:
- k1 = 1.2
- b = 0.75

Tuned params (change if you want):
- k1 = 1.6
- b = 0.55

Outputs:
- outputs/run_bm25_default.csv
- outputs/run_bm25_tuned.csv


In [ ]:
DEFAULT_K1, DEFAULT_B = 1.2, 0.75
TUNED_K1, TUNED_B = 1.6, 0.55

bm25_default = BM25(corpus_df['text'].tolist(), k1=DEFAULT_K1, b=DEFAULT_B)
bm25_tuned = BM25(corpus_df['text'].tolist(), k1=TUNED_K1, b=TUNED_B)

run_default = run_bm25(bm25_default, corpus_df['doc_id'].tolist(), queries_df, topk=100)
run_tuned = run_bm25(bm25_tuned, corpus_df['doc_id'].tolist(), queries_df, topk=100)

run_default.to_csv(OUT_DIR / 'run_bm25_default.csv', index=False)
run_tuned.to_csv(OUT_DIR / 'run_bm25_tuned.csv', index=False)

print('Saved:', OUT_DIR / 'run_bm25_default.csv')
print('Saved:', OUT_DIR / 'run_bm25_tuned.csv')

# If qrels is missing, create a starter pool (top-20 per query) to label
if not QRELS_PATH.exists():
    qrels_df = build_qrels_pool(run_default, topn=20)
    qrels_df.to_csv(QRELS_PATH, index=False)
    print('Created starter qrels pool at:', QRELS_PATH)
    print('Now open data/qrels.csv and label relevance as 0, 1, or 2 for each row.')

run_default.head(10)


### Parameter effect check (length and rank movement)

Use these prints to write your Part A explanation.


In [ ]:
doc_len = dict(zip(corpus_df['doc_id'], corpus_df['text'].map(lambda x: len(tokenize(x)))))

def avg_len_topk(run_df, k=10):
    lens = []
    for qid, grp in run_df.groupby('query_id'):
        top = grp.sort_values('rank').head(k)['doc_id'].tolist()
        lens.extend([doc_len.get(d, 0) for d in top])
    return float(np.mean(lens)) if lens else 0.0

avg_len_def = avg_len_topk(run_default, 10)
avg_len_tun = avg_len_topk(run_tuned, 10)
print('Avg doc length in top-10, default:', round(avg_len_def, 2))
print('Avg doc length in top-10, tuned  :', round(avg_len_tun, 2))

sample_qid = queries_df['query_id'].iloc[0]

def ranks_for_query(run_df, qid):
    return {row.doc_id: int(row.rank) for row in run_df[run_df['query_id']==qid].itertuples(index=False)}

rdef = ranks_for_query(run_default, sample_qid)
rtun = ranks_for_query(run_tuned, sample_qid)

movement = []
for did in set(rdef.keys()) | set(rtun.keys()):
    rd = rdef.get(did, 9999)
    rt = rtun.get(did, 9999)
    movement.append((did, rd, rt, rd - rt))  # positive means moved up in tuned

move_df = pd.DataFrame(movement, columns=['doc_id','rank_default','rank_tuned','moved_up_by']).sort_values('moved_up_by', ascending=False)
move_df.head(15)


### Part A written analysis (human style)

Write 6 to 10 lines in your own voice, using your numbers above.  
What to cover:
- What changed between default and tuned
- Which doc type got helped or hurt (long vs short, repetition, etc.)
- Why that matches BM25 math (k1 saturation and b length normalization)

Tip:
- Lower b means less length penalty.
- Higher k1 means more weight for repeated query terms before saturation.


## Part B - Evaluation with P@10, MAP, nDCG@10

Make sure `data/qrels.csv` is filled with your labels before running Part B.  
If you just created the starter pool, edit it first, then re-run this cell.

Output:
- outputs/metrics_summary.csv


In [ ]:
# Reload qrels if you edited it
qrels_df = pd.read_csv(QRELS_PATH)

def precision_at_k(run_df, qrels_df, k=10):
    qrels = qrels_df.copy()
    qrels['is_rel'] = qrels['relevance'] > 0
    rel_lookup = {(r.query_id, r.doc_id): bool(r.is_rel) for r in qrels.itertuples(index=False)}

    scores = []
    for qid, grp in run_df.groupby('query_id'):
        top = grp.sort_values('rank').head(k)
        hits = sum(1 for did in top['doc_id'] if rel_lookup.get((qid, did), False))
        scores.append(hits / k)

    return float(np.mean(scores)) if scores else 0.0

def average_precision_and_by_query(run_df, qrels_df):
    qrels = qrels_df.copy()
    qrels['is_rel'] = qrels['relevance'] > 0
    rel_lookup = {(r.query_id, r.doc_id): bool(r.is_rel) for r in qrels.itertuples(index=False)}
    total_rel = qrels.groupby('query_id')['is_rel'].sum().to_dict()

    ap_by_q = {}
    for qid, grp in run_df.groupby('query_id'):
        grp = grp.sort_values('rank')
        num_rel = 0
        sum_prec = 0.0
        for i, did in enumerate(grp['doc_id'], start=1):
            if rel_lookup.get((qid, did), False):
                num_rel += 1
                sum_prec += num_rel / i

        denom = total_rel.get(qid, 0)
        ap_by_q[qid] = (sum_prec / denom) if denom > 0 else 0.0

    MAP = float(np.mean(list(ap_by_q.values()))) if ap_by_q else 0.0
    return MAP, ap_by_q

def ndcg_at_k(run_df, qrels_df, k=10):
    rel_grade = {(r.query_id, r.doc_id): int(r.relevance) for r in qrels_df.itertuples(index=False)}
    ndcgs = []

    for qid, grp in run_df.groupby('query_id'):
        grp = grp.sort_values('rank').head(k)
        gains = [rel_grade.get((qid, did), 0) for did in grp['doc_id']]

        dcg = 0.0
        for i, g in enumerate(gains, start=1):
            dcg += (2**g - 1) / math.log2(i + 1)

        ideal = qrels_df[qrels_df['query_id'] == qid]['relevance'].astype(int).tolist()
        ideal = sorted(ideal, reverse=True)[:k]

        idcg = 0.0
        for i, g in enumerate(ideal, start=1):
            idcg += (2**g - 1) / math.log2(i + 1)

        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    return float(np.mean(ndcgs)) if ndcgs else 0.0

p10_def = precision_at_k(run_default, qrels_df, 10)
p10_tun = precision_at_k(run_tuned, qrels_df, 10)

map_def, ap_def = average_precision_and_by_query(run_default, qrels_df)
map_tun, ap_tun = average_precision_and_by_query(run_tuned, qrels_df)

nd_def = ndcg_at_k(run_default, qrels_df, 10)
nd_tun = ndcg_at_k(run_tuned, qrels_df, 10)

metrics_df = pd.DataFrame([
    ('bm25_default', p10_def, map_def, nd_def),
    ('bm25_tuned', p10_tun, map_tun, nd_tun),
], columns=['method', 'P@10', 'MAP', 'nDCG@10'])

metrics_df.to_csv(OUT_DIR / 'metrics_summary.csv', index=False)
print('Saved:', OUT_DIR / 'metrics_summary.csv')
metrics_df


### Part B written analysis (human style)

Use your computed metrics and answer:
- Which run is better overall
- Which queries improved and which got worse (use AP per query, ap_def and ap_tun)
- What MAP captures that P@10 does not

Short idea to say:
- P@10 only cares about top 10.
- MAP cares about ranking of all relevant docs, so it punishes if relevant docs show up late.


## Part C - Precision Recall Curve

Pick one query id and make the PR curve using the default run.

Outputs:
- outputs/pr_curve_query_<query_id>.csv
- outputs/pr_curve_query_<query_id>.png


In [ ]:
SELECTED_QUERY_ID = queries_df['query_id'].iloc[0]  # change if you want

def pr_points_for_query(run_df, qrels_df, qid):
    rel_bin = {(r.query_id, r.doc_id): int(r.relevance > 0) for r in qrels_df.itertuples(index=False)}
    total_rel = sum(v for (qq, _), v in rel_bin.items() if qq == qid)

    rows = []
    num_rel = 0

    ranked = run_df[run_df['query_id'] == qid].sort_values('rank')
    for _, r in ranked.iterrows():
        did = r['doc_id']
        is_rel = rel_bin.get((qid, did), 0)
        if is_rel == 1:
            num_rel += 1

        rank = int(r['rank'])
        prec = num_rel / rank
        rec = num_rel / total_rel if total_rel > 0 else 0.0

        rows.append((qid, rank, did, int(is_rel), float(prec), float(rec)))

    return pd.DataFrame(rows, columns=['query_id', 'rank', 'doc_id', 'is_relevant', 'precision', 'recall'])

pr_df = pr_points_for_query(run_default, qrels_df, SELECTED_QUERY_ID)
pr_csv = OUT_DIR / f'pr_curve_query_{SELECTED_QUERY_ID}.csv'
pr_png = OUT_DIR / f'pr_curve_query_{SELECTED_QUERY_ID}.png'

pr_df.to_csv(pr_csv, index=False)

plt.figure()
plt.plot(pr_df['recall'], pr_df['precision'])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve for {SELECTED_QUERY_ID} (BM25 default)')
plt.grid(True, alpha=0.3)
plt.savefig(pr_png, dpi=200, bbox_inches='tight')
plt.close()

print('Saved:', pr_csv)
print('Saved:', pr_png)
pr_df.head(10)


### Part C written analysis (human style)

Answer:
- Does precision drop fast or slow as recall grows
- What does it say about early ranking vs later ranking

Common reading:
- Big early drop means your top ranks are clean but later results are mostly noise.
- Slow drop means you keep finding relevant docs deeper in the ranking.


## Part D - Error Analysis

We pick the worst query based on AP from the default run and generate a short writeup.
Then you can open outputs/error_analysis.txt and adjust wording if needed.

Output:
- outputs/error_analysis.txt


In [ ]:
_, ap_def = average_precision_and_by_query(run_default, qrels_df)
worst_qid = min(ap_def.items(), key=lambda x: x[1])[0] if ap_def else SELECTED_QUERY_ID

rel_grade = {(r.query_id, r.doc_id): int(r.relevance) for r in qrels_df.itertuples(index=False)}

top10 = run_default[run_default['query_id'] == worst_qid].sort_values('rank').head(10)
wrong_high = [d for d in top10['doc_id'].tolist() if rel_grade.get((worst_qid, d), 0) == 0][:5]

relevant_docs = qrels_df[(qrels_df['query_id'] == worst_qid) & (qrels_df['relevance'] > 0)]['doc_id'].tolist()
rank_map = {r.doc_id: int(r.rank) for r in run_default[run_default['query_id'] == worst_qid].itertuples(index=False)}

missed_or_low = []
for d in relevant_docs:
    rnk = rank_map.get(d, None)
    if rnk is None or rnk > 10:
        missed_or_low.append((d, rnk))
missed_or_low = missed_or_low[:5]

def snippet(doc_id, n=180):
    txt = corpus_df.loc[corpus_df['doc_id'] == doc_id, 'text'].values
    if len(txt) == 0:
        return ''
    s = str(txt[0]).replace('\n', ' ').strip()
    return (s[:n] + '...') if len(s) > n else s

lines = []
lines.append(f'Poor query picked: {worst_qid}')
lines.append('')
lines.append('What went wrong:')
lines.append(f'- AP (default BM25): {ap_def.get(worst_qid, 0.0):.4f}')
lines.append('')

lines.append('Docs ranked high but judged not relevant (examples):')
for d in wrong_high:
    lines.append(f'- {d}: {snippet(d)}')
lines.append('')

lines.append('Relevant docs missed or pushed low (examples):')
for d, rnk in missed_or_low:
    rnk_txt = 'not retrieved in top 100' if rnk is None else f'rank {rnk}'
    lines.append(f'- {d}: {rnk_txt} | {snippet(d)}')
lines.append('')

lines.append('One concrete improvement to try next:')
lines.append('- Add light normalization for term variants (stemming or lemmatization), then re-run BM25. If the issue is vocabulary mismatch, also test adding a couple of synonyms to the query.')

error_path = OUT_DIR / 'error_analysis.txt'
error_path.write_text('\n'.join(lines), encoding='utf-8')

print('Saved:', error_path)
print('\n'.join(lines[:25]))


## Final checklist before you zip

1) Confirm corpus has at least 400 docs.
2) Confirm you have 10 queries (or explain if fewer).
3) Confirm qrels has exactly the pooled top-20 per query and each row has 0, 1, or 2.
4) Confirm all output CSV headers match the assignment.
5) Zip the folder with the required name format.

